<a href="https://colab.research.google.com/github/yoonkim313/dataCampusProject-Team10/blob/master/Relation%20Extraction/5.AutomatedRelationExtraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!pip install transformers
!apt-get update
!apt-get install g++ openjdk-8-jdk 
!pip3 install konlpy
!pip install kss
%cd /content/drive/Shared drives/BigDATA TEAM 10/OpenInformationExtraction
!pip install soykeyword
from frameBERT import frame_parser
path="/content/drive/Shared drives/BigDATA TEAM 10/OpenInformationExtraction"

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive
     |████████████████████████████████| 778kB 5.6MB/s 
     |████████████████████████████████| 1.1MB 30.1MB/s 
     |████████████████████████████████| 890kB 24.7MB/s 
     |████████████████████████████████| 3.0MB 35.8MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=049b1a6f7c9150629e0e72a9ff984cd502719d2a6faca365a9e8944bb592d293
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a5012363

In [ ]:
import kss
from konlpy.tag import Hannanum, Okt
import re
import time
import numpy as np
import heapq
import pandas as pd
from operator import itemgetter
from collections import deque
from ast import literal_eval
from collections import defaultdict
from soykeyword.lasso import LassoKeywordExtractor
from pprint import pprint
!pip install krwordrank
from krwordrank.word import KRWordRank
from copy import deepcopy
h = Hannanum()
okt = Okt()

In [ ]:
text = '''
평등은 자유와 더불어 근대 사회의 핵심 이념으로 자리 잡고있다. 인간은 가령 인종이나 성별과 상관없이 누구나 평등하다고 생각한다. 모든 인간은 평등하다고 말하는데, 이 말은 무슨 뜻일까? 그리고 그 근거는 무엇인가? 
일단 이 말을 모든 인간을 모든 측면에서 똑같이 대우하는 절대칙 평등으로 생각하는 이는 없다. 인간은 저마다 다르게 가지고 대어난 능력과 소질을 똑같게 만들 수 없기 때문이다. 절대적 평등은 개인의 개성이냐 자율성 등의 가치와 충돌하기도 한다. 평등에 대한 요구는 모든 불평등을 악으로 보는 것이 아니라 충분한 이유가 제시되지 않은 불평등을 제거하는 데 목표를 두고 있다. ‘이유 없는 차별 금지’라는 조건적 평등 원칙은 차별 대우를 할 때는 이유를 제시할 것을 요구하고 있다. 이것은 어떤 이유가 제시된다면 특정한 부류에 속하는 사람들에게는 평등한 대우를, 그 부류에 속하지 않는 사람들에게는 차별저 대우를 하는 것을 허용한다. 그렇다면 사람들을 특정한 부류로 구분하는 기준은 무엇인가? 이것은 바로 평등의 근거에 대한 물음이다.
근대의 여러 인권 선언에 나타난 평등 개념은 개인들 사이의 평등성을 타고난 자연적 권리로 간주하였다. 하지만 이러한 자연권 이론은 무엇이 자연적 권리이고 권리의 존재가 자명한 이유가 무엇인지 등의 문제에 부딪히게 된다. 그래서 롤스는 기존의 자연권 사상에 의존하지 않는 방시으로 인간 평등의 근거를 마런하려고 한다. 그는 어떤 규칙이 공평하고 일관되게 운영되며, 그 규칙에 따라 유사한 경우는 유사하게 취급된다면 형식적 정의는 실현된다고 본다. 하지만 롤스는 형식저 정의에 따라 규칙을 준수하는 것만으로는 정의를 담보할 수 없다고 생각한다. 그 규칙이 더 높은 도덕적 권위를 지닌 다른 이넘과 충돌할 수 었기에, 실질적 정의가 보장되기 위해서는 규칙의 내용이 중요한 것이다.
롤스는 인간 평등의 근거를 설명하면서 영역 성질 (range property) 개념을 도입한다. 예를 들어 어떤 원의 내부에 있는 점들은 그 위치가 서로 다르지만 원의 내부에 있다는 점에서 동일한 영역 성질을 갖는다. 반면에 원의 내부에 있는 집과 원의 외부에 였는 점은원의 경계선을 기준으로 서로 다른 영역 성질을 갖는다. 그는 평등한 대우를 받기 위한 영역 성질로서 ‘도덕적 인격'을 제시한다. 도덕적 인격이란 도덕적 호소가 가능하고 그런 호소에 관심을 기이는 능력이 있다는 것인데, 이 능력을 최소치만 갖고 있다면 평등한 대우에 대한 권한을 갖게 된다. 도덕적 인격이라고 해서 도덕적으로 훌륭하다는 뜻이 아니라 도덕과 무관하다는 말과 대비되는 뜻으로 쓰고 있다. 그런데 어린 아이는 인격체로서의 최소한의 기준을 충족하고 있는지가 논란이 될 수 있다. 이에 대해 롤스는 도덕적 인격을 규정하는 최소한의 요구 조건은 잠재적 능력이지 그것의 실현 여부가 아니기에 어린 아이도 평등한 존재라고 말한다. 싱어는 위와 같은 롤스의 시도를 비판한다. 도덕에 대한 민감성의 수준은 사람에 따라 다르다. 그래서 도덕적 인격의 능력이 그렇게 중요하다면 그것을 갖춘 정도에 따라 도덕적 위계를 다르게 하지 말아야 할 이유가 분명하지 않다고 말한다. 
'''

In [43]:
class Text:
  def __init__(self, text):
    self.text = text
    text = re.sub('\n',' ',text)
    text = re.sub("'",' ',text)
    paragraphs = text.split('\n')
    self.docs = [kss.split_sentences(paragraph) for paragraph in paragraphs]

  def add_tags(self, tag):
    
    for i in range(len(self.idx)):
      idx = [(self.newtext.find(" "+cand), len(cand)) for cand in self.candidates if self.newtext.find(cand)!=-1]
      print(idx)
      indices = (idx[i][0],idx[i][0]+idx[i][1]+1)
      word = self.newtext[indices[0]+1:indices[1]]
      print(word)
      tagged = "<%s>%s</%s>" % (tag, word, tag)
      print(tagged)
      self.newtext = tagged.join([self.newtext[:indices[0]], self.newtext[indices[1]:]])
    return self.newtext

  def keywords(self):
    wordrank_extractor = KRWordRank(min_count=4, max_length=10)
    keywords, rank, graph = wordrank_extractor.extract(documents, num_keywords=50)
    p=[]
    kw=[]
    for k, v in keywords.items():
      p.append(okt.pos(k))
    for ls in p:
      for tup in ls:
          print(tup[0],tup[1])
          if tup[1].startswith("N"):
            kw.append(tup[0])
    return kw
    conj_tag = "conj"  # html 만들 떄 mark 부분을 conj로 변경하기

class Highlight(Text):
  def __init__(self, text):
    conj = '그리고, 그런데, 그러나, 그래도, 그래서, 또는, 및, 즉, 게다가, 따라서, 때문에, 아니면, 왜냐하면, 단, 오히려, 비록'
    conj = conj.replace("'","")
    conj = conj.replace(", "," ")
    self.text = text
    self.candidates = conj.split(" ")
    self.idx = [(self.text.find(" "+cand), len(cand)) for cand in self.candidates if self.text.find(cand)!=-1]
    self.newtext = deepcopy(self.text)


c.add_tags(conj_tag)


도덕 Noun
적 Suffix
평등 Noun
인간 Noun
이유 Noun
가 Josa
능력 Noun
따라 Verb
있다 Adjective
. Punctuation
어떤 Adjective
영역 Noun
대우 Noun
근거 Noun
롤스 Noun
는 Josa
모든 Noun
원 Noun
의 Josa
규칙 Noun
존재 Noun
인격 Noun
있는 Adjective
성질 Noun
다르 Adjective
대한 Noun
권리 Noun
제시 Noun
정의 Noun
무엇 Noun
자연 Noun
최소 Noun
사람 Noun
['도덕', '평등', '인간', '이유', '능력', '영역', '대우', '근거', '롤스', '모든', '원', '규칙', '존재', '인격', '성질', '대한', '권리', '제시', '정의', '무엇', '자연', '최소', '사람']


In [ ]:
def main():
  t = Text(text)
  print(t.keywords())

In [19]:
pprint(okt.tagset)
print()
pprint(h.tagset)

{'Adjective': '형용사',
 'Adverb': '부사',
 'Alpha': '알파벳',
 'Conjunction': '접속사',
 'Determiner': '관형사',
 'Eomi': '어미',
 'Exclamation': '감탄사',
 'Foreign': '외국어, 한자 및 기타기호',
 'Hashtag': '트위터 해쉬태그',
 'Josa': '조사',
 'KoreanParticle': '(ex: ㅋㅋ)',
 'Noun': '명사',
 'Number': '숫자',
 'PreEomi': '선어말어미',
 'Punctuation': '구두점',
 'ScreenName': '트위터 아이디',
 'Suffix': '접미사',
 'Unknown': '미등록어',
 'Verb': '동사'}

{'E': '어미',
 'EC': '연결 어미',
 'EF': '종결 어미',
 'EP': '선어말어미',
 'ET': '전성 어미',
 'F': '외국어',
 'I': '독립언',
 'II': '감탄사',
 'J': '관계언',
 'JC': '격조사',
 'JP': '서술격 조사',
 'JX': '보조사',
 'M': '수식언',
 'MA': '부사',
 'MM': '관형사',
 'N': '체언',
 'NB': '의존명사',
 'NC': '보통명사',
 'NN': '수사',
 'NP': '대명사',
 'NQ': '고유명사',
 'P': '용언',
 'PA': '형용사',
 'PV': '동사',
 'PX': '보조 용언',
 'S': '기호',
 'X': '접사',
 'XP': '접두사',
 'XS': '접미사'}


#### 하지만, 그런데 등의 연결사에 해당하는 부분 추출

css 문법
<html>
<head>
<style>
mark { 
  background-color: yellow;
  color: black;
}
</style>
</head>
<body>

<p>A mark element is displayed like this:</p>

<mark>Highlighted text!!</mark>

<p>Change the default CSS settings to see the effect.</p>

</body>
</html>


In [22]:
conj = '그리고, 그런데, 그러나, 그래도, 그래서, 또는, 및, 즉, 게다가, 따라서, 때문에, 아니면, 왜냐하면, 단, 오히려, 비록'
conj = conj.replace("'","")
conj = conj.replace(", "," ")
candidates = conj.split(" ")
print(candidates)
idx = [(text.find(cand), len(cand)) for cand in candidates if text.find(cand)!=-1]
print(idx)

['그리고', '그런데', '그러나', '그래도', '그래서', '또는', '및', '즉', '게다가', '따라서', '때문에', '아니면', '왜냐하면', '단', '오히려', '비록']
[(107, 3), (1292, 3), (656, 3), (1766, 1), (1850, 3), (124, 1)]


In [36]:
conj_tag = "conj"  # html 만들 떄 mark 부분을 conj로 변경하기

class Highlight:
  def __init__(self, text):
    conj = '그리고, 그런데, 그러나, 그래도, 그래서, 또는, 및, 즉, 게다가, 따라서, 때문에, 아니면, 왜냐하면, 단, 오히려, 비록'
    conj = conj.replace("'","")
    conj = conj.replace(", "," ")
    self.text = text
    self.candidates = conj.split(" ")
    self.idx = [(self.text.find(" "+cand), len(cand)) for cand in self.candidates if self.text.find(cand)!=-1]
    self.newtext = deepcopy(self.text)

  def add_tags(self, tag):
    
    for i in range(len(self.idx)):
      idx = [(self.newtext.find(" "+cand), len(cand)) for cand in self.candidates if self.newtext.find(cand)!=-1]
      print(idx)
      indices = (idx[i][0],idx[i][0]+idx[i][1]+1)
      word = self.newtext[indices[0]+1:indices[1]]
      print(word)
      tagged = "<%s>%s</%s>" % (tag, word, tag)
      print(tagged)
      self.newtext = tagged.join([self.newtext[:indices[0]], self.newtext[indices[1]:]])
    return self.newtext

c = Highlight(text)
c.add_tags(conj_tag)

[(106, 3), (1291, 3), (655, 3), (1765, 1), (1849, 3), (-1, 1)]
그리고
<conj>그리고</conj>
[(1564, 3), (1303, 3), (667, 3), (1777, 1), (1861, 3), (-1, 1)]
그런데
<conj>그런데</conj>
[(1576, 3), (-1, 3), (667, 3), (1789, 1), (1873, 3), (-1, 1)]
그래서
<conj>그래서</conj>
[(1588, 3), (-1, 3), (1509, 3), (1801, 1), (1885, 3), (-1, 1)]
즉
<conj>즉</conj>
[(1588, 3), (-1, 3), (1509, 3), (-1, 1), (1897, 3), (-1, 1)]
때문에
<conj>때문에</conj>
[(1588, 3), (-1, 3), (1509, 3), (-1, 1), (2004, 3), (-1, 1)]
 
<conj> </conj>


' 평등은 자유와 더불어 근대 사회의 핵심 이념으로 자리 잡고 있다. 인간은 가령 인종이나 성별과 상관없이 누구나 평등하다고 생각한다. 모든 인간은 평등하다고 말하는데, 이 말은 무슨 뜻일까?<conj>그리고</conj> 그 근거는 무엇인가? 일단 이 말을 모든 인간을 모든 측면에서 똑같이 대우하는 절대칙 평등으로 생각하는 이는 없다. 인간은 저마다 다르게 가지고 대어난 능력과 소질을 똑같게 만들 수 없기 때문이다. 절대적 평등은 개인의 개성이냐 자율성 등의 가치와 충돌하기도 한다. 평등에 대한 요구는 모든 불평등을 악으로 보는 것이 아니라 충분한 이유가 제시되지 않은 불평등을 제거하는 데 목표를 두고 있다. ‘이유 없는 차별 금지’라는 조건적 평등 원칙은 차별 대우를 할 때는 이유를 제시할 것을 요구하고 있다. 이것은 어떤 이유가 제시된다면 특정한 부류에 속하는 사람들에게는 평등한 대우를, 그 부류에 속하지 않는 사람들에게는 차별저 대우를 하는 것을 허용한다. 그렇다면 사람들을 특정한 부류로 구분하는 기준은 무엇인가? 이것은 바로 평등의 근거에 대한 물음이다. 근대의 여러 인권 선언에 나타난 평등 개념은 개인들 사이의 평등성을 타고난 자연적 권리로 간주하였다. 하지만 이러한 자연권 이론은 무엇이 자연적 권리이고 권리의 존재가 자명한 이유가 무엇인지 등의 문제에 부딪히게 된다.<conj>그래서</conj> 롤스는 기존의 자연권 사상에 의존하지 않는 방시으로 인간 평등의 근거를 마런 하려고 한다. 그는 어떤 규칙이 공평하고 일관되게 운영되며, 그 규칙에 따라 유사한 경우는 유사하게 취급된다면 형식적 정의는 실현된다고 본다. 하지만 롤스는 형식저 정의에 따라 규칙을 준수하는 것만으로는 정의를 담보할 수 없다고 생각한다. 그 규칙이 더 높은 도덕적 권위를 지닌 다른 이넘과 충돌할 수 었기에, 실질적 정의가 보장되기 위해서는 규칙의 내용이 중요한 것이다. 롤스는 인간 평등의 근거를 설명하면서 영역 성질 (range property) 개념을 도입한다. 예를 들어 어떤 

In [30]:
ls = []
ls.append(h.pos(text))
m = []
for i in ls:
  for word, pos in i:
    if pos=="M":
      m.append(word)

print(m)

['가령', '상관없이', '모든', '이', '무슨', '그리고', '그', '이', '모든', '모든', '똑같이', '모든', '어떤', '그', '그렇다면', '바로', '여러', '그래서', '어떤', '그', '그', '더', '다른', '어떤', '그', '서로', '서로', '다른', '이', '그런데', '그래서', '그리고', '그래서', '어떤', '즉', '그리고', '그', '더', '그', '아무런']


## Relation Extraction

In [ ]:
parser = frame_parser.FrameParser(model_path=path, language='ko')

srl model: framenet
language: ko
version: 1.2
using viterbi: False
using masking: True
pretrained BERT: bert-base-multilingual-cased
using TGT special token: True
used dictionary:
	 /content/drive/Shared drives/BigDATA TEAM 10/OpenInformationExtraction/frameBERT/src/../koreanframenet/resource/info/kfn1.2_lu2idx.json
	 /content/drive/Shared drives/BigDATA TEAM 10/OpenInformationExtraction/frameBERT/src/../koreanframenet/resource/info/kfn1.2_lufrmap.json
	 /content/drive/Shared drives/BigDATA TEAM 10/OpenInformationExtraction/frameBERT/src/../koreanframenet/resource/info/mul_bio_frargmap.json
...loaded model path: /content/drive/Shared drives/BigDATA TEAM 10/OpenInformationExtraction
/content/drive/Shared drives/BigDATA TEAM 10/OpenInformationExtraction
...model is loaded


### Linked List Implemented to Show the whole list of sentences and their parsed tags in a consecutive manner

In [ ]:
def split_into_sentences(object):
  ls = kss.split_sentences(object)
  return ls

def frameParse(text):
  parser = frame_parser.FrameParser(model_path=path, language='ko')
  parsed = parser.parser(text, sent_id='1', result_format='conll')
  return parsed
  

def sortCandidate(parsed, num_candidates):
  for i in range(len(parsed)):
    count=0
    for element in parsed[i][3]:
      if element.startswith("O"):
        count+=1
  q[i] = len(words) - count
  hq = heapq.nlargest(num_candidates, q)
  indices.append(q.index(hq[idx]))
  # for idx in range(num_candidates):
  #   temp = parsed[q.index(hq[idx])]
  #   words = temp[0]
  #   [[row[i] for row in temp] for i in range(len(words))]
  return indices


def findConsecutiveBIO(words, tag):
  began = False
  count = 0
  q = deque(words)
  for i in range(len(words)):
    if tag[i]=='O':
      q.popleft()
    if tag[i].startswith('B'):
      began=True
    if began & tag[i].startswith('I'):
      count+=1
  return q

# Linked List 구현해서 parsed 후보군에 사용
class Node:
    def __init__(self, data):
        self.words = data[0]
        self.tags = data[3]
        self.next = None

    def __repr__(self):
        return str((self.words, self.tags))

class LinkedList:
  def __init__(self):
      self.head = None

  def __repr__(self):
      node = self.head
      nodes = []
      while node is not None:
          nodes.append(str(node.tags))
          node = node.next
      nodes.append("None")
      return ' -> '.join(nodes)
  
class LinkedList:
  def __init__(self):
      self.head = None

  def __repr__(self):
      node = self.head
      nodes = []
      while node is not None:
          nodes.append(str(node.tags))
          node = node.next
      nodes.append("None")
      return ' -> '.join(nodes)

def extractFrame(text):
  ls = split_into_sentences(text)
  final = {}
  
  for idx in range(len(ls)):
    parsed = frameParse(ls[idx]) # candidates 생성
    final.setdefault(idx,str)
    parsedList =  LinkedList()
    for j in range(len(parsed)):
      parsed_candidate = parsed[j]
      new_node = Node(parsed_candidate)
      if j == 0:
        old_node = new_node
        parsedList.head = old_node
      elif j==len(parsed)-1:
        old_node.next = new_node
        new_node.next = None
        print(idx,'  ',parsedList)
        final[idx] = parsedList
      else:
        old_node.next = new_node
        old_node = new_node
  return final



In [ ]:
def main(text):
  all = extractFrame(text)
  s = ""
  res = {}

  i = 1
  res.setdefault(i,tuple)
  for k, v in all.items():
    a = v.head
    while a is not None:
      temp = " "
      if len(a.words) == len(a.tags):
        q = findConsecutiveBIO(a.words, a.tags)
      else:
        print("ERROR OCCURED")
        
      count = len(a.words)
      for tag in a.tags:
       if tag.startswith("O"):
          count -= 1
      if count > len(a.words)*0.5:
        print("The threshold is reached !! 😛 😝 😜 🤪 🤨 🧐 🤓 😎 🤩 🥳 😏")
        temp = " ".join(q) 
        print(temp)
        words = a.words
        tags = a.tags
      res[i] = (a.words, a.tags , " ".join(q))
      a = a.next
    i+=1
  return res


In [ ]:
extracted = main(text)
pprint(extracted)

srl model: framenet
language: ko
version: 1.2
using viterbi: False
using masking: True
pretrained BERT: bert-base-multilingual-cased
using TGT special token: True
used dictionary:
	 /content/drive/Shared drives/BigDATA TEAM 10/OpenInformationExtraction/frameBERT/src/../koreanframenet/resource/info/kfn1.2_lu2idx.json
	 /content/drive/Shared drives/BigDATA TEAM 10/OpenInformationExtraction/frameBERT/src/../koreanframenet/resource/info/kfn1.2_lufrmap.json
	 /content/drive/Shared drives/BigDATA TEAM 10/OpenInformationExtraction/frameBERT/src/../koreanframenet/resource/info/mul_bio_frargmap.json
...loaded model path: /content/drive/Shared drives/BigDATA TEAM 10/OpenInformationExtraction
/content/drive/Shared drives/BigDATA TEAM 10/OpenInformationExtraction
...model is loaded


/content/drive/Shared drives/BigDATA TEAM 10/OpenInformationExtraction/frameBERT/src/utils.py:279: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  pred_logits = sm(masked_logit).view(1,-1)


0    ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'] -> ['O', 'O', 'O', 'O', 'O', 'O', 'I-Class', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'] -> ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'] -> ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-Time', 'O', 'O', 'O', 'O', 'O'] -> ['O', 'O', 'O', 'I-Landmark_event', 'I-Landmark_event', 'I-Landmark_event', 'I-Landmark_event', 'I-Landmark_event', 'I-Landmark_event', 'I-Landmark_event', 'O', 'O', 'I-Trajector_event', 'I-Trajector_event', 'O', 'I-Trajector_event'] -> ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'] -> ['O', 'O', 'O', 'I-Time', 'I-Time', 'I-Time', 'I-Time', 'I-Time', 'I-Time', 'I-Time', 'I-Time', 'B-Old', 'O', 'O', 'O', 'O'] -> ['O', 'O', 'I-Means', 'O', 'O', 'O', 'O', 'O', 'O', 'I-Time', 'O', 'I-Goal', 'I-Goal', 'O', 'O', 'O'] -> None
srl model: framenet
language: ko
version: 1.2
using viterbi: False
using masking: True
pr

In [ ]:
pprint(extracted)

{1: (['신체의',
      '세포,',
      '조직,',
      '장기가',
      '손상되어',
      '더',
      '이상',
      '제',
      '기능을',
      '하지못할',
      '때에',
      '이를',
      '대체하기',
      '위해',
      '이식을',
      '실시한다.'],
     ['O',
      'O',
      'I-Means',
      'O',
      'O',
      'O',
      'O',
      'O',
      'O',
      'I-Time',
      'O',
      'I-Goal',
      'I-Goal',
      'O',
      'O',
      'O'],
     ' '),
 2: (['이때', '이식으로', '옮겨', '붙이는', '세포,', '조직,', '장기를', '이식편이라', '한다.'],
     ['B-Time', 'B-Goal', 'I-Goal', 'O', 'B-Items', 'O', 'O', 'O', 'O'],
     ' '),
 3: (['자신이나',
      '일란성',
      '쌍둥이의',
      '이식편을',
      '이용할',
      '',
      '수',
      '없다면',
      '다른',
      '사람의',
      '이식편으로',
      '‘동종',
      '이식’을',
      '실시한다.'],
     ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'],
     ' ')}


### TODO : 여기서부터 extracted를 가지고 purpose / goal / effect / entity 등을 찾아야함

In [ ]:
for k, tup in extracted.items():
  a, b, c = tup
  roles = [0]*len(b)
  words = [0]*len(b)
  for i in range(len(b)):
    try: 
      roles[i] = b[i].split("-")[1] 
      
    except: 
      roles[i] = b[i].split("-")[0]
  print(roles)
  for _ in roles:
    pass

In [ ]:
# 1st sentence : GOALS & MEANS 

parsed = parser.parser(ls[0], sent_id='1', result_format='conll')
words = parsed[0][0]
print("😀 Words vector from sentence ",words)
q = [-1]*len(parsed)
for i in range(len(parsed)):
  count=0
  for element in parsed[i][3]:
    if element.startswith("O"):
      count+=1
  q[i] = len(words) - count
hq = heapq.nlargest(2, q)


print("Number of parsed candidates ",len(parsed))
print(q, heapq.nlargest(2, q)) # 첫번째 경우 사용


words = parsed[0][0]
roles = parsed[0][2]
tagged = parsed[0][3]
for idx in range(len(tagged)):
  try:
    roles[i] = tagged[i].split("-")[1]
  except:
    roles[i] = tagged[i].split("-")[0]

print("roles ",roles)

goals = []
means = []
for _ in range(len(words)):
  if roles[_]=="Goal":
    goals.append(words[_])
  if roles[_]=='Means':
    means.append(words[_])

# MEANS
MEANS = ' '.join(means)
pos = h.pos(MEANS)
imp = []
for tup in h.pos(MEANS):
  if tup[1]=="N":
    imp.append(tup[0])
MEANS_TRUNCATED = ' '.join(imp)
print(MEANS_TRUNCATED)

# GOALS
GOAL = ' '.join(goals)
pos = h.pos(GOAL)
imp = []
for tup in h.pos(GOAL):
  if tup[1]=="N":
    imp.append(tup[0])
GOAL_TRUNCATED = ' '.join(imp)
print(GOAL_TRUNCATED)

final_string = MEANS_TRUNCATED + ' -- >' + GOAL_TRUNCATED
print("😀 final string to be inserted",final_string)

[['TEXT2PPTX의', '구현하기', '위해', 'two', 'track', 'process를', '거쳤습니다.'], ['_', '_', '위하다.v', '_', '_', '_', '_'], ['_', '_', 'Purpose', '_', '_', '_', '_'], ['B-Goal', 'I-Goal', 'O', 'O', 'I-Means', 'I-Means', 'I-Means']]


In [ ]:
# 2nd sentence : PURPOSE & INSTRUMENT
parsed = parser.parser(ls[1], sent_id='1', result_format='conll')
words = parsed[0][0]
print("😀 Words vector from sentence ",words)
q = [-1]*len(parsed)
for i in range(len(parsed)):
  count=0
  for element in parsed[i][3]:
    if element.startswith("O"):
      count+=1
  q[i] = len(words) - count


print("Number of parsed candidates ",len(parsed))
print(q, heapq.nlargest(3, q))
print(itemgetter(*[1,5])(parsed))
candidates = itemgetter(*[1,8])(parsed)
for _ in candidates:
  words = _[0]
  tagged = _[3]
  print("tagged vector of candidates: ",tagged)
  roles = _[2]
  for i in range(len(parsed)):
    try:
      roles[i] = tagged[i].split("-")[1]
    except:
      roles[i] = tagged[i].split("-")[0]
  print(roles,f'Tagged words for the roles : {tagged}')
# roles are selected as the one with purpose and instrument

purpose = []
instrument = []
for _ in range(len(words)):
  if roles[_]=="Purpose":
    purpose.append(words[_])
    
  elif roles[_]=="Instrument":
    instrument.append(words[_])
    
  elif roles[_]!="_": # using 포함
    instrument.append(words[_])
print("Instrument: ",instrument)
print("Purpose: ",purpose)





# PURPOSE
PURPOSE = ' '.join(purpose)
pos = h.pos(PURPOSE)
imp = []
for tup in pos:
  if tup[1]=="N":
    imp.append(tup[0])
PURPOSE_TRUNCATED = ' '.join(imp)
print(PURPOSE_TRUNCATED)

# INSTRUMENT
INSTRUMENT = ' '.join(instrument)
pos = h.pos(INSTRUMENT)
imp = []
for tup in pos:
  if tup[1]=="N":
    imp.append(tup[0])
INST_TRUNCATED = ' '.join(imp)
print(INST_TRUNCATED)

final_string = INST_TRUNCATED+ ' ➡️' + PURPOSE_TRUNCATED 
print("😀 final string to be inserted ", final_string)

    
    

/content/drive/Shared drives/BigDATA TEAM 10/OpenInformationExtraction/frameBERT/src/utils.py:279: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  pred_logits = sm(masked_logit).view(1,-1)


😀 Words vector from sentence  ['대본을', '피피티로', '옮기기', '전에', '요약하고', '분석하기', '위해', '자연어처리의', '최신', '기술을', '다수', '사용하였습니다.']
Number of parsed candidates  9
[2, 11, 4, 0, 6, 1, 1, 1, 11] [11, 11, 6]
([['대본을', '피피티로', '옮기기', '전에', '요약하고', '분석하기', '위해', '자연어처리의', '최신', '기술을', '다수', '사용하였습니다.'], ['_', '_', '_', '전.n', '_', '_', '_', '_', '_', '_', '_', '_'], ['_', '_', '_', 'Time_vector', '_', '_', '_', '_', '_', '_', '_', '_'], ['B-Landmark_event', 'I-Landmark_event', 'I-Landmark_event', 'O', 'B-Event', 'I-Event', 'I-Event', 'I-Event', 'I-Event', 'I-Event', 'I-Event', 'I-Event']], [['대본을', '피피티로', '옮기기', '전에', '요약하고', '분석하기', '위해', '자연어처리의', '최신', '기술을', '다수', '사용하였습니다.'], ['_', '_', '_', '_', '_', '_', '_', '_', '최신.n', '_', '_', '_'], ['_', '_', '_', '_', '_', '_', '_', '_', 'Relative_time', '_', '_', '_'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-Focal_participant', 'O', 'O']])
tagged vector of candidates:  ['B-Landmark_event', 'I-Landmark_event', 'I-Landmark_event', 'O', 'B-Event

In [ ]:
# 3rd sentence : Goal - Means
parsed = parser.parser(ls[2], sent_id='1', result_format='conll')
words = parsed[0][0]
print("Words vector from sentence ",words)
q = [-1]*len(parsed)
for i in range(len(parsed)):
  count=0
  for element in parsed[i][3]:
    if element.startswith("O"):
      count+=1
  q[i] = len(words) - count

print(q, heapq.nlargest(3, q))

a,b,c=parsed
tagged = b[3]
for i in range(len(tagged)):
  try:
    roles[i] = tagged[i].split("-")[1]
  except:
    roles[i] = tagged[i].split("-")[0]



goals = []
means = []
for _ in range(len(words)):
  if roles[_]=="Goal":
    goals.append(words[_])
  if roles[_]=='Means':
    means.append(words[_])

# MEANS
MEANS = ' '.join(means)
pos = h.pos(MEANS)
imp = []
for tup in h.pos(MEANS):
  if tup[1]=="N":
    imp.append(tup[0])
MEANS_TRUNCATED = ' '.join(imp)
print(MEANS_TRUNCATED)

# GOALS
GOAL = ' '.join(goals)
pos = h.pos(GOAL)
imp = []
for tup in h.pos(GOAL):
  if tup[1]=="N":
    imp.append(tup[0])
GOAL_TRUNCATED = ' '.join(imp)
print(GOAL_TRUNCATED)

final_string = MEANS_TRUNCATED + ' -- >' + GOAL_TRUNCATED
print("😀 final string to be inserted",final_string)


/content/drive/Shared drives/BigDATA TEAM 10/OpenInformationExtraction/frameBERT/src/utils.py:279: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  pred_logits = sm(masked_logit).view(1,-1)


Words vector from sentence  ['또한', '파이썬에서', '파워포인트의', '소스에', '접근하기', '위해', 'xml', '코드를', '심층적으로', '분석했습니다.']
[2, 8, 3] [8, 3, 2]
askdf
askdf
askdf
askdf
   
코드 심층적 분석
파이썬 파워포인트 소스 접근하기
😀 final string to be inserted 코드 심층적 분석-->파이썬 파워포인트 소스 접근하기


## PPTX에 활용

In [ ]:
prs = Presentation("final_template.pptx")  # 원하는 template 종류 불러오기
slide_0 = prs.slides[0] # 제복 slide (제목 슬라이드)
slide_2 = prs.slides[2] # 두 번째 slide

In [ ]:
slide_2.placeholders.element[3][2][2][1][1].text = "Two Track Process" # 여덟 번째 슬라이드 제목
slide_2.placeholders.element[5][2][2][1][1].text = ""
slide_2.placeholders.element[6][2][2][1][1].text = ""
slide_2.placeholders.element[17][2][2][1][1].text = ""
slide_2.placeholders.element[18][2][2][1][1].text = ""
slide_2.placeholders.element[19][2][2][1][1].text = "TEXT2PPTX"
slide_2.placeholders.element[11][2][2][1][1].text = "자연어처리의 최신 기술을 다수 사용"
slide_2.placeholders.element[10][2][2][1][1].text = "대본을 피피티로 옮기기 전에 요약하고 분석"
slide_2.placeholders.element[14][2][2][1][1].text = "파이썬에서 파워포인트의 소스에 접근하기 위해 xml 코드를 심층적으로 분석"
slide_2.placeholders.element[13][2][2][1][1].text = "사용자가 자연어로 쓰인 대본을 텍2피에 제공하면 높은 수준의 피피티로 제공"

## 교육용 교재에 활용